In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

headers: dict = {
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Accept-Language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
    'Cookie': "JSESSIONID=714DEE803F60F70A0D22D568A513660B.app1; Path=/frx; Secure; HttpOnly; SameSite=None",
    'Referer': 'https://www.rexel.fr/frx/Cat%C3%A9gorie/R%C3%A9seau-informatique/Produit-cuivre/Prise-et-connecteur-de-type-RJ45-pour-r%C3%A9seau-LAN/Actassi-S-One-connecteur-RJ45-cat%C3%A9gorie-6-blind%C3%A9/SCHVDIB17726B12/p/69617534',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest',
    'sec-ch-ua': '"Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";v="120"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': "macOS"
}

def get_price(ids: str) -> dict:
    url: str = f'https://www.rexel.fr/frx/erp/getPrice.json?products={ids}'
    response: dict = requests.get(url, headers=headers)
    
    return response.json()

def scrap_data(html: BeautifulSoup, k_products: int = 10) -> pd.DataFrame:
    df: pd.DataFrame = pd.DataFrame(
        data=[],
        columns=['id', 'category', 'brand', 'name', 'image-url']
    )
    product_divs: list = html.find_all(attrs='product-list-item')[:k_products]

    for idx, product_div in enumerate(product_divs):
        df.loc[idx] = {
            'id': product_div.get('id'),
            'category': product_div.get('data-category'),
            'brand': product_div.get('data-productbrand'),
            'name': product_div.get('data-product-name'),
            'image-url': product_div.get('data-product-image-url'),
        }
    df: pd.DataFrame = df.query('category.notna()').reset_index().drop('index', axis=1)

    # Query items price
    product_ids: str = df['id'].str.cat(sep="|")
    df['price_HT']: pd.Series = [p['price'] for p in get_price(product_ids)]
    
    return df

def search_products(query: str) -> pd.DataFrame:
    url_website: str = f"https://www.rexel.fr/frx/search?text={query}"
    html: BeautifulSoup = BeautifulSoup(requests.get(url_website, headers=headers).content, 'html.parser')

    return scrap_data(html)

In [ ]:
query: str = "Cable USB C"
search_products(query)